# Deep Learning Project: Adversarial Attacks

This notebook implements and evaluates adversarial attacks (FGSM, PGD) on a ResNet-34 model using a subset of ImageNet. It is converted from a Python script.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import json
import os
import shutil
from PIL import Image
import matplotlib.pyplot as plt

# Magic command for matplotlib in notebooks
%matplotlib inline


## 1. Configuration

Define base paths, dataset locations, and result directories. **Important:** Verify and adjust `base_project_path` and `standard_imagenet_index_file_path` if necessary for your environment.

In [ ]:
base_project_path = "/home/ubuntu/project_files"
dataset_path = os.path.join(base_project_path, "dataset/TestDataSet")
standard_imagenet_index_file_path = os.path.join(base_project_path, "imagenet_class_index.json")
results_dir = os.path.join(base_project_path, "results")
adv_dataset1_path = os.path.join(base_project_path, "AdversarialTestSet1")
adv_dataset2_path = os.path.join(base_project_path, "AdversarialTestSet2") # For Task 3
visualizations_dir_task2 = os.path.join(results_dir, "visualizations_task2")
visualizations_dir_task3 = os.path.join(results_dir, "visualizations_task3") # For Task 3

# Ensure project_files and subdirectories exist
for path in [base_project_path, results_dir, adv_dataset1_path, adv_dataset2_path, visualizations_dir_task2, visualizations_dir_task3]:
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


print(f"Base project path: {base_project_path}")
print(f"Dataset path: {dataset_path}")
print(f"ImageNet index file: {standard_imagenet_index_file_path}")
print(f"Results directory: {results_dir}")
print("
Checking and creating directories...")
for path_to_check in [base_project_path, results_dir, adv_dataset1_path, adv_dataset2_path, visualizations_dir_task2, visualizations_dir_task3]:
    if not os.path.exists(path_to_check):
        os.makedirs(path_to_check, exist_ok=True)
        print(f"Created directory: {path_to_check}")
    else:
        print(f"Directory already exists: {path_to_check}")
print("Directory setup complete.")


## 2. Helper Functions

Utility functions for image transformations, denormalization, ImageNet class index mapping, and model evaluation.

### 2.1 Normalization Constants and Device Setup

In [ ]:
mean_norms = np.array([0.485, 0.456, 0.406])
std_norms = np.array([0.229, 0.224, 0.225])

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


### 2.2 Image Transformations

In [ ]:
def get_plain_transforms():
    return transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean_norms, std=std_norms)
    ])


print("Helper function 'get_plain_transforms' defined.")

### 2.3 Denormalization

In [ ]:
def denormalize(tensor_image):
    img = tensor_image.clone().detach().cpu()
    for t, m, s in zip(img, mean_norms, std_norms):
        t.mul_(s).add_(m)
    img = img.clamp(0, 1)
    return transforms.ToPILImage()(img)


print("Helper function 'denormalize' defined.")

### 2.4 ImageNet Index Mapping

In [ ]:
def get_idx_to_imagenet_idx_map(current_dataset, imagenet_json_path):
    idx_map = {}
    try:
        with open(imagenet_json_path, 'r') as f:
            standard_idx_to_synset_map = json.load(f)
        synset_to_standard_idx_map = {}
        for std_idx_str, data_list in standard_idx_to_synset_map.items():
            synset_id = data_list[0]
            synset_to_standard_idx_map[synset_id] = int(std_idx_str)
        
        for folder_synset_id, internal_folder_idx in current_dataset.class_to_idx.items():
            if folder_synset_id in synset_to_standard_idx_map:
                standard_imagenet_idx = synset_to_standard_idx_map[folder_synset_id]
                idx_map[str(internal_folder_idx)] = standard_imagenet_idx
            else:
                print(f"Warning: Dataset folder synset_id {folder_synset_id} not found in standard ImageNet mapping.")
        
        if not idx_map or len(idx_map) != len(current_dataset.classes):
            print(f"Warning: Mapping issues. Mapped {len(idx_map)} of {len(current_dataset.classes)} classes.")
        else:
            print(f"Successfully created mapping for {len(idx_map)} classes.")
        return idx_map
    except Exception as e:
        print(f"Error creating index map: {e}")
        return {}


# Note: Ensure 'imagenet_class_index.json' is available at 'standard_imagenet_index_file_path'.
# If not, you might need to download it. Example command (run in a separate cell if needed):
# !mkdir -p {os.path.dirname(standard_imagenet_index_file_path)} # Ensure parent directory exists
# !wget https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json -O {standard_imagenet_index_file_path}
print("Helper function 'get_idx_to_imagenet_idx_map' defined.")


### 2.5 Model Evaluation

In [ ]:
def evaluate_model(model, dataloader, device, idx_to_imagenet_idx, model_name="Model"):
    model.eval()
    correct_top1 = 0
    correct_top5 = 0
    total_eval_samples = 0

    if not idx_to_imagenet_idx:
        print(f"Error in evaluate_model for {model_name}: idx_to_imagenet_idx is empty.")
        return 0.0, 0.0

    with torch.no_grad():
        for inputs, internal_labels in dataloader:
            inputs, internal_labels = inputs.to(device), internal_labels.to(device)
            outputs = model(inputs)
            
            _, predicted_top1_indices = torch.max(outputs, 1)
            _, predicted_top5_indices_k = torch.topk(outputs, 5, dim=1)

            for i in range(internal_labels.size(0)):
                internal_folder_idx = internal_labels[i].item()
                actual_target_imagenet_idx = idx_to_imagenet_idx.get(str(internal_folder_idx))

                if actual_target_imagenet_idx is None:
                    continue 
                
                total_eval_samples +=1
                predicted_imagenet_idx_top1 = predicted_top1_indices[i].item()
                if predicted_imagenet_idx_top1 == actual_target_imagenet_idx:
                    correct_top1 += 1
                
                predicted_imagenet_indices_top5 = predicted_top5_indices_k[i].tolist()
                if actual_target_imagenet_idx in predicted_imagenet_indices_top5:
                    correct_top5 += 1
    
    if total_eval_samples == 0:
        print(f"Warning for {model_name}: Total samples for evaluation was 0.")
        return 0.0, 0.0
                    
    top1_accuracy = 100 * correct_top1 / total_eval_samples
    top5_accuracy = 100 * correct_top5 / total_eval_samples
    print(f"{model_name} - Top-1 Accuracy: {top1_accuracy:.2f}%, Top-5 Accuracy: {top5_accuracy:.2f}% on {total_eval_samples} samples")
    return top1_accuracy, top5_accuracy


print("Helper function 'evaluate_model' defined.")

## 3. Task 1: Baseline Evaluation

Evaluate the pre-trained ResNet-34 model on the original (clean) TestDataSet.

### 3.1 Define `run_task1`

In [ ]:
def run_task1(model, device, current_dataset, idx_to_imagenet_idx):
    print("\n--- Running Task 1: Baseline Evaluation ---")
    dataloader = torch.utils.data.DataLoader(current_dataset, batch_size=32, shuffle=False, num_workers=0)
    top1_acc, top5_acc = evaluate_model(model, dataloader, device, idx_to_imagenet_idx, "ResNet-34 Baseline")
    
    results_file_task1 = os.path.join(results_dir, "task1_results.txt")
    result_text = f"Task 1: Baseline ResNet-34 Performance\n"
    result_text += f"Top-1 Accuracy: {top1_acc:.2f}%\n"
    result_text += f"Top-5 Accuracy: {top5_acc:.2f}%\n"
    with open(results_file_task1, "w") as f:
        f.write(result_text)
    print(f"Task 1 results saved to {results_file_task1}")
    return top1_acc, top5_acc


print("Function 	'run_task1' defined.")

## 4. Task 2: Pixel-wise Attack (FGSM)

Implement the Fast Gradient Sign Method (FGSM) attack and evaluate the model's performance on the generated adversarial examples.

### 4.1 Define `fgsm_attack`

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    # The L_inf constraint is that ||perturbed_image - image||_inf <= epsilon.
    # This is ensured by the construction if epsilon is the step size.
    # However, to be absolutely sure and to keep image values in valid range for normalized images (approx [-2, 2]),
    # it's good practice to clamp the *perturbation* itself to [-epsilon, epsilon] then add to original image,
    # or clamp the final perturbed_image to [image_orig - epsilon, image_orig + epsilon].
    # For FGSM, the formula is direct. The problem states "L∞ distance between new and original is no greater than ε = 0.02"
    # This means ||x_adv - x||_inf <= eps. Our construction x_adv = x + eps * sign(grad) implies ||x_adv - x||_inf = ||eps * sign(grad)||_inf = eps * ||sign(grad)||_inf = eps * 1 = eps.
    # So the constraint is met by definition of FGSM.
    return perturbed_image


print("Function 	'fgsm_attack' defined.")

### 4.2 Define `run_task2`

In [ ]:
def run_task2(model, device, original_dataset, idx_to_imagenet_idx, epsilon=0.02):
    print("\n--- Running Task 2: FGSM Attack ---")
    model.eval()
    loss_fn = torch.nn.CrossEntropyLoss()
    
    if os.path.exists(adv_dataset1_path):
        shutil.rmtree(adv_dataset1_path)
    os.makedirs(adv_dataset1_path, exist_ok=True)

    for class_name in original_dataset.classes:
        os.makedirs(os.path.join(adv_dataset1_path, class_name), exist_ok=True)

    num_visualized = 0
    max_visualizations = 5
    total_images_processed = 0
    original_dataloader = torch.utils.data.DataLoader(original_dataset, batch_size=1, shuffle=False, num_workers=0)

    for i, (data, internal_target_idx) in enumerate(original_dataloader):
        data, internal_target_idx = data.to(device), internal_target_idx.to(device)
        data.requires_grad = True

        output = model(data)
        initial_pred_idx = output.max(1, keepdim=True)[1].item()
        
        true_imagenet_idx = idx_to_imagenet_idx.get(str(internal_target_idx.item()))
        if true_imagenet_idx is None:
            print(f"Skipping image {i} in Task 2 due to missing label mapping for internal idx {internal_target_idx.item()}")
            continue
        
        target_for_loss = torch.tensor([true_imagenet_idx], device=device)
        loss = loss_fn(output, target_for_loss)
        model.zero_grad()
        loss.backward()
        data_grad = data.grad.data

        perturbed_data = fgsm_attack(data.detach(), epsilon, data_grad) # Detach data before modification

        # Verify L-infinity distance
        l_inf_dist_check = torch.norm(perturbed_data - data, p=float('inf')).item()
        if l_inf_dist_check > epsilon + 1e-5: # Check with tolerance
             print(f"Warning Task 2: L_inf distance {l_inf_dist_check} for image {i} exceeded epsilon {epsilon}.")

        with torch.no_grad():
            output_adv = model(perturbed_data)
        final_pred_idx = output_adv.max(1, keepdim=True)[1].item()

        original_img_path, _ = original_dataset.samples[i]
        class_name = os.path.basename(os.path.dirname(original_img_path))
        filename = os.path.basename(original_img_path)
        
        adv_img_pil = denormalize(perturbed_data.squeeze(0))
        adv_img_pil.save(os.path.join(adv_dataset1_path, class_name, filename))
        total_images_processed += 1

        if num_visualized < max_visualizations and final_pred_idx != true_imagenet_idx and initial_pred_idx == true_imagenet_idx:
            original_pil = denormalize(data.squeeze(0).detach())
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(original_pil)
            plt.title(f"Original (T2): Pred {initial_pred_idx}\nTrue: {true_imagenet_idx}")
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(adv_img_pil)
            plt.title(f"FGSM (eps={epsilon}): Pred {final_pred_idx}\nTrue: {true_imagenet_idx}")
            plt.axis('off')
            vis_filename = f"fgsm_visualization_{num_visualized+1}.png"
            plt.savefig(os.path.join(visualizations_dir_task2, vis_filename))
            plt.close()
            print(f"Saved Task 2 visualization: {vis_filename}")
            num_visualized += 1
        
        if (i+1) % 50 == 0:
            print(f"Processed {i+1}/{len(original_dataset)} images for FGSM attack.")

    print(f"FGSM attack complete. {total_images_processed} adversarial images saved to {adv_dataset1_path}")
    adv_dataset1_eval = torchvision.datasets.ImageFolder(root=adv_dataset1_path, transform=get_plain_transforms())
    adv_dataloader1 = torch.utils.data.DataLoader(adv_dataset1_eval, batch_size=32, shuffle=False, num_workers=0)
    adv_idx_to_imagenet_idx1 = get_idx_to_imagenet_idx_map(adv_dataset1_eval, standard_imagenet_index_file_path)
    
    top1_acc_adv, top5_acc_adv = evaluate_model(model, adv_dataloader1, device, adv_idx_to_imagenet_idx1 if adv_idx_to_imagenet_idx1 else idx_to_imagenet_idx, "ResNet-34 on AdvSet1 (FGSM)")
    results_file_task2 = os.path.join(results_dir, "task2_results.txt")
    result_text = f"Task 2: FGSM Attack (epsilon={epsilon}) Performance\nTop-1 Accuracy: {top1_acc_adv:.2f}%\nTop-5 Accuracy: {top5_acc_adv:.2f}%\n"
    with open(results_file_task2, "w") as f: f.write(result_text)
    print(f"Task 2 results saved to {results_file_task2}")
    return top1_acc_adv, top5_acc_adv


print("Function 	'run_task2' defined.")

## 5. Task 3: Improved Attack (PGD/I-FGSM)

Implement the Projected Gradient Descent (PGD) attack, an iterative version of FGSM, and evaluate the model.

### 5.1 Define `pgd_attack`

In [ ]:
def pgd_attack(model, image_original, true_imagenet_label, epsilon, alpha, num_iter, device):
    # PGD is an iterative version of FGSM
    # image_original should be the original clean image, detached and requires_grad=False
    # We create a new tensor for the adversarial image that requires grad
    perturbed_image = image_original.clone().detach().to(device)
    perturbed_image.requires_grad = True
    loss_fn = torch.nn.CrossEntropyLoss()

    for _ in range(num_iter):
        output = model(perturbed_image)
        loss = loss_fn(output, torch.tensor([true_imagenet_label], device=device))
        model.zero_grad()
        loss.backward()
        
        with torch.no_grad():
            data_grad = perturbed_image.grad.data
            # FGSM step
            perturbed_image_step = perturbed_image + alpha * data_grad.sign()
            # Project perturbation back to epsilon ball around original image
            perturbation = torch.clamp(perturbed_image_step - image_original, -epsilon, epsilon)
            perturbed_image = image_original + perturbation
            # Optional: Clamp to valid image range if necessary (e.g. [0,1] for unnormalized, or model's expected normalized range)
            # For normalized images, this usually means clamping to approx [-min_val/std, (1-min_val)/std]
            # For now, we assume the model handles values resulting from this process.
            # The L_inf constraint is ||perturbed_image - image_original||_inf <= epsilon, which is enforced by the clamp.
        # perturbed_image.grad.zero_() # Zero gradients for next iteration - REMOVED TO FIX AttributeError
        perturbed_image = perturbed_image.detach().clone() # Detach and clone for next iteration's requires_grad
        perturbed_image.requires_grad = True

    return perturbed_image.detach() # Return final detached adversarial image


print("Function 	'pgd_attack' defined.")

### 5.2 Define `run_task3`

In [ ]:
def run_task3(model, device, original_dataset, idx_to_imagenet_idx, epsilon=0.02, alpha_factor=1.25, num_iter=10):
    print("\n--- Running Task 3: Improved Attack (PGD/I-FGSM) ---")
    model.eval()
    alpha = epsilon / num_iter * alpha_factor # Step size, often a bit larger than eps/steps

    if os.path.exists(adv_dataset2_path):
        shutil.rmtree(adv_dataset2_path)
    os.makedirs(adv_dataset2_path, exist_ok=True)

    for class_name in original_dataset.classes:
        os.makedirs(os.path.join(adv_dataset2_path, class_name), exist_ok=True)

    num_visualized = 0
    max_visualizations = 5
    total_images_processed = 0
    original_dataloader = torch.utils.data.DataLoader(original_dataset, batch_size=1, shuffle=False, num_workers=0)

    for i, (data_orig, internal_target_idx) in enumerate(original_dataloader):
        data_orig = data_orig.to(device) # Original clean image, no grad needed here for PGD func
        internal_target_idx = internal_target_idx.to(device)

        with torch.no_grad():
            output_orig = model(data_orig)
        initial_pred_idx = output_orig.max(1, keepdim=True)[1].item()

        true_imagenet_idx = idx_to_imagenet_idx.get(str(internal_target_idx.item()))
        if true_imagenet_idx is None:
            print(f"Skipping image {i} in Task 3 due to missing label mapping for internal idx {internal_target_idx.item()}")
            continue

        perturbed_data = pgd_attack(model, data_orig, true_imagenet_idx, epsilon, alpha, num_iter, device)
        
        # Verify L-infinity distance
        l_inf_dist_check = torch.norm(perturbed_data - data_orig, p=float('inf')).item()
        if l_inf_dist_check > epsilon + 1e-5: # Check with tolerance
             print(f"Warning Task 3: L_inf distance {l_inf_dist_check} for image {i} exceeded epsilon {epsilon}.")

        with torch.no_grad():
            output_adv = model(perturbed_data)
        final_pred_idx = output_adv.max(1, keepdim=True)[1].item()

        original_img_path, _ = original_dataset.samples[i]
        class_name = os.path.basename(os.path.dirname(original_img_path))
        filename = os.path.basename(original_img_path)
        
        adv_img_pil = denormalize(perturbed_data.squeeze(0))
        adv_img_pil.save(os.path.join(adv_dataset2_path, class_name, filename))
        total_images_processed += 1

        if num_visualized < max_visualizations and final_pred_idx != true_imagenet_idx and initial_pred_idx == true_imagenet_idx:
            original_pil = denormalize(data_orig.squeeze(0).detach())
            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(original_pil)
            plt.title(f"Original (T3): Pred {initial_pred_idx}\nTrue: {true_imagenet_idx}")
            plt.axis('off')
            plt.subplot(1, 2, 2)
            plt.imshow(adv_img_pil)
            plt.title(f"PGD (eps={epsilon}, iter={num_iter}): Pred {final_pred_idx}\nTrue: {true_imagenet_idx}")
            plt.axis('off')
            vis_filename = f"pgd_visualization_{num_visualized+1}.png"
            plt.savefig(os.path.join(visualizations_dir_task3, vis_filename))
            plt.close()
            print(f"Saved Task 3 visualization: {vis_filename}")
            num_visualized += 1

        if (i+1) % 50 == 0:
            print(f"Processed {i+1}/{len(original_dataset)} images for PGD attack.")

    print(f"PGD attack complete. {total_images_processed} adversarial images saved to {adv_dataset2_path}")
    adv_dataset2_eval = torchvision.datasets.ImageFolder(root=adv_dataset2_path, transform=get_plain_transforms())
    adv_dataloader2 = torch.utils.data.DataLoader(adv_dataset2_eval, batch_size=32, shuffle=False, num_workers=0)
    adv_idx_to_imagenet_idx2 = get_idx_to_imagenet_idx_map(adv_dataset2_eval, standard_imagenet_index_file_path)

    top1_acc_adv, top5_acc_adv = evaluate_model(model, adv_dataloader2, device, adv_idx_to_imagenet_idx2 if adv_idx_to_imagenet_idx2 else idx_to_imagenet_idx, "ResNet-34 on AdvSet2 (PGD)")
    results_file_task3 = os.path.join(results_dir, "task3_results.txt")
    result_text = f"Task 3: Improved Attack (PGD, epsilon={epsilon}, iters={num_iter}) Performance\nTop-1 Accuracy: {top1_acc_adv:.2f}%\nTop-5 Accuracy: {top5_acc_adv:.2f}%\n"
    with open(results_file_task3, "w") as f: f.write(result_text)
    print(f"Task 3 results saved to {results_file_task3}")
    return top1_acc_adv, top5_acc_adv


print("Function 	'run_task3' defined.")

## 6. Main Execution / Orchestration

This section provides cells to load the model, dataset, and run all tasks sequentially. Execute these cells in order. Ensure `base_project_path`, `dataset_path`, and `standard_imagenet_index_file_path` are correctly set in the Configuration section (Cell 4).

### 6.1 Load Pre-trained Model and Original Dataset

In [ ]:

# --- Main Execution Logic ---
print("Setting up device...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

print("
Loading pre-trained ResNet-34 model...")
model = torchvision.models.resnet34(weights=torchvision.models.ResNet34_Weights.IMAGENET1K_V1)
model.to(device)
model.eval() # Set model to evaluation mode
print("ResNet-34 model loaded and moved to device.")

print("
Loading original dataset...")
# Ensure the dataset_path points to your TestDataSet folder
if not os.path.exists(dataset_path) or not os.listdir(dataset_path):
    print(f"ERROR: Original dataset not found or empty at {dataset_path}")
    print("Please ensure your TestDataSet (e.g., from ILSVRC2012 validation set, or a subset) is correctly placed.")
    print("The script expects subfolders named by synset ID (e.g., n01440764) containing images.")
    # Create dummy dataset and idx_map to prevent later errors if user wants to proceed partially
    original_dataset = None
    idx_to_imagenet_idx = {}
else:
    plain_transforms = get_plain_transforms()
    original_dataset = torchvision.datasets.ImageFolder(root=dataset_path, transform=plain_transforms)
    print(f"Loaded {len(original_dataset)} images from {dataset_path}")
    print(f"Dataset classes (folder names): {original_dataset.classes[:5]}... (first 5)") # Show a few class names

    print("
Creating mapping from dataset internal indices to ImageNet indices...")
    idx_to_imagenet_idx = get_idx_to_imagenet_idx_map(original_dataset, standard_imagenet_index_file_path)
    if not idx_to_imagenet_idx:
        print("Warning: Index mapping failed. Evaluation results will be unreliable or zero.")
    else:
        print(f"Index mapping created for {len(idx_to_imagenet_idx)} classes.")

# Check if dataset loaded properly before proceeding
if original_dataset and idx_to_imagenet_idx:
    print("
Model, dataset, and index mapping loaded successfully.")
else:
    print("
Critical error: Model, dataset, or index mapping could not be loaded. Subsequent tasks may fail or produce incorrect results.")


### 6.2 Run Task 1: Baseline Evaluation

In [ ]:

if original_dataset and idx_to_imagenet_idx:
    print("
--- Attempting to run Task 1 ---")
    task1_top1_acc, task1_top5_acc = run_task1(model, device, original_dataset, idx_to_imagenet_idx)
    print(f"Task 1 Complete. Top-1: {task1_top1_acc:.2f}%, Top-5: {task1_top5_acc:.2f}%")
else:
    print("Skipping Task 1 due to issues with dataset loading or index mapping.")


### 6.3 Run Task 2: FGSM Attack

In [ ]:

if original_dataset and idx_to_imagenet_idx:
    print("
--- Attempting to run Task 2 ---")
    # You can adjust epsilon here if needed
    fgsm_epsilon = 0.02 
    task2_top1_acc, task2_top5_acc = run_task2(model, device, original_dataset, idx_to_imagenet_idx, epsilon=fgsm_epsilon)
    print(f"Task 2 Complete. FGSM (eps={fgsm_epsilon}) Top-1: {task2_top1_acc:.2f}%, Top-5: {task2_top5_acc:.2f}%")
    print(f"Adversarial images for Task 2 saved in: {adv_dataset1_path}")
    print(f"Visualizations for Task 2 saved in: {visualizations_dir_task2}")
else:
    print("Skipping Task 2 due to issues with dataset loading or index mapping.")


### 6.4 Run Task 3: PGD Attack

In [ ]:

if original_dataset and idx_to_imagenet_idx:
    print("
--- Attempting to run Task 3 ---")
    # You can adjust PGD parameters here if needed
    pgd_epsilon = 0.02
    pgd_alpha_factor = 1.25 # alpha = epsilon / num_iter * alpha_factor
    pgd_num_iter = 10
    task3_top1_acc, task3_top5_acc = run_task3(model, device, original_dataset, idx_to_imagenet_idx, epsilon=pgd_epsilon, alpha_factor=pgd_alpha_factor, num_iter=pgd_num_iter)
    print(f"Task 3 Complete. PGD (eps={pgd_epsilon}, iter={pgd_num_iter}) Top-1: {task3_top1_acc:.2f}%, Top-5: {task3_top5_acc:.2f}%")
    print(f"Adversarial images for Task 3 saved in: {adv_dataset2_path}")
    print(f"Visualizations for Task 3 saved in: {visualizations_dir_task3}")
else:
    print("Skipping Task 3 due to issues with dataset loading or index mapping.")
